In [1]:
#Generate INSERT statements for MySQL**
import pandas as pd
import mysql.connector

In [3]:
#Generate INSERT statements for MySQL**
import pandas as pd
import mysql.connector

# Database configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Chelah@123',  
    'database': 'trade_data'
}

# Connect to database to get foreign key mappings
def get_foreign_key_mappings():
    """Get foreign key mappings from database"""
    try:
        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor()
        
        mappings = {}
        
        # Get regions
        cursor.execute("SELECT region_name, region_id FROM regions")
        mappings['regions'] = {name: id for name, id in cursor.fetchall()}
        
        # Get countries
        cursor.execute("SELECT country_name, country_id FROM countries")
        mappings['countries'] = {name: id for name, id in cursor.fetchall()}
        
        # Get commodity categories
        cursor.execute("SELECT category_name, category_id FROM commodity_categories")
        mappings['categories'] = {name: id for name, id in cursor.fetchall()}
        
        # Get commodity subcategories
        cursor.execute("SELECT subcategory_name, subcategory_id FROM commodity_subcategories")
        mappings['subcategories'] = {name: id for name, id in cursor.fetchall()}
        
        # Get COVID periods
        cursor.execute("SELECT period_name, period_id FROM covid_periods")
        mappings['covid_periods'] = {name: id for name, id in cursor.fetchall()}
        
        # Get economic events
        cursor.execute("SELECT event_name, event_id FROM economic_events")
        mappings['economic_events'] = {name: id for name, id in cursor.fetchall()}
        
        # Get trade partners
        cursor.execute("SELECT country_id, partner_id FROM trade_partners")
        mappings['trade_partners'] = {country_id: partner_id for country_id, partner_id in cursor.fetchall()}
        
        cursor.close()
        connection.close()
        return mappings
        
    except mysql.connector.Error as err:
        print(f"Database connection failed: {err}")
        return None

#Load dataset**
df = pd.read_csv("data1.csv")

# Get foreign key mappings from database
mappings = get_foreign_key_mappings()

trade_records_inserts = []

for index, row in df.iterrows():
    # Get foreign key IDs
    country_id = mappings['countries'].get(row['Partner_Country'])
    partner_id = mappings['trade_partners'].get(country_id)
    category_id = mappings['categories'].get(row['Commodity_Category'])
    subcategory_id = mappings['subcategories'].get(row['Commodity_Subcategory'])
    covid_period_id = mappings['covid_periods'].get(row['COVID_Period'])
    economic_event_id = mappings['economic_events'].get(row['Economic_Event'])
    
    # Trade Records table insert
    trade_records_inserts.append(f"""INSERT INTO trade_records (year, month, quarter, trade_type, partner_id, category_id, subcategory_id, value_usd_million, value_kes_billion, volume_metric_tons, growth_rate_yoy, covid_period_id, economic_event_id) VALUES ({int(row['Year'])}, '{row['Month']}', '{row['Quarter']}', '{row['Trade_Type']}', {partner_id}, {category_id}, {subcategory_id}, {row['Value_USD_Million']}, {row['Value_KES_Billion']}, {row['Volume_Metric_Tons']}, {row['Growth_Rate_YoY']}, {covid_period_id}, {economic_event_id});""")

#Save to .sql file**
output_file = "trade_data_inserts.sql"
with open(output_file, "w") as f:
    f.write("-- Inserts for trade_records\n")
    f.write("\n".join(trade_records_inserts))

print(f"SQL insert statements saved to {output_file}")

SQL insert statements saved to trade_data_inserts.sql
